<h2> The Basics of Flask Architecture</h2>
<ul>
<li>the app object routes the HTML to directories to handle HTML, AJAX, and PHP Requests</li>
<li>In HTML: href="testHref" --> in Python @app.route('/testHref')</li>
<li>In AJAX: url:'/testURL' --> in Python @app.route('/testURL') will handle the AJAX request and can be recieved with the request object. This is basically the same as post and is used with all inputs as keys and their contents as values.  </li>
<li>Use the example to sync the MySQL with the app</li>
<li>Use a MySQL.connect().cursor().call_proc('yourProcedure', (item1, item2, item3)) // assuming its an insert procedure</li>
<li>https://code.tutsplus.com/tutorials/creating-a-web-app-from-scratch-using-python-flask-and-mysql--cms-22972</li>
</ul>

In [37]:
from flask import Flask, render_template, request, json
# on the fly ipython notebook package installer
#!pip install flask-mysql
from flask.ext.mysql import MySQL
from werkzeug import generate_password_hash, check_password_hash

app = Flask(__name__)


In [38]:
@app.route('/showSignUp')
def showSignUp():
    return render_template('signup.html')

@app.route('/signUp',methods=['POST'])
def signUp():
 
    # read the posted values from the UI
    _name = request.form['inputName']
    _email = request.form['inputEmail']
    _password = request.form['inputPassword']
 
    # validate the received values
    if _name and _email and _password:
        #return json.dumps({'html':'<span>All fields good !!</span>'})
        
        mysql = MySQL()
 
        # MySQL configurations
        app.config['MYSQL_DATABASE_USER'] = 'root'
        app.config['MYSQL_DATABASE_PASSWORD'] = 'YOUR KNOW THE PW'
        app.config['MYSQL_DATABASE_DB'] = 'BucketList'
        app.config['MYSQL_DATABASE_HOST'] = 'localhost'
        mysql.init_app(app)


        # establish mysql connection
        conn = mysql.connect()
        cursor = conn.cursor()
        _hashed_password = generate_password_hash(_password)
        cursor.callproc('sp_createUser',(_name,_email,_hashed_password))


        data = cursor.fetchall() 
        if len(data) is 0:
            conn.commit()
            return json.dumps({'message':'User created successfully !'})
        else:
            return json.dumps({'error':str(data[0])})
        
    else:
        return json.dumps({'html':'<span>Enter the required fields</span>'})
    
    

    
@app.route("/")
def main():
    return render_template('index.html')

In [ ]:
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2017 17:54:02] "GET / HTTP/1.1" 200 -
